In [1]:
# !git clone https://github.com/ultralytics/yolov5

In [1]:
!cd yolov5 & pip install -r requirements.txt

In [2]:
import torch
import os
from IPython.display import Image, clear_output

In [3]:
!cd yolov5
yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/best.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\Gokul/.cache\torch\hub\master.zip
YOLOv5  2022-3-7 torch 1.10.2+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [4]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import pytesseract
from PIL import Image

In [5]:
import os
import numpy as np
import pandas as pd
import cv2
import imutils
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import math
from sklearn.metrics import f1_score

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers

from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Input, Dropout
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [6]:
def rotation_func(img):
    #Step 1: contours
    cnts = cv2.findContours(img.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
    cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30] #sort contours based on their area keeping minimum required area as '30' (anything smaller than this will not be considered)
    NumberPlateCnt = None #we currently have no Number plate contour

    # loop over our contours to find the best possible approximate contour of number plate
    count = 0
    for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            if len(approx) == 4:  # Select the contour with 4 corners
                NumberPlateCnt = approx #This is our approx Number Plate Contour
                x,y,w,h = cv2.boundingRect(c)
                ROI = img[y:y+h, x:x+w]
                break

    if NumberPlateCnt is not None:
        # Drawing the selected contour on the original image
        cv2.drawContours(img, [NumberPlateCnt], -1, (0,255,0), 3)
        
#     print(NumberPlateCnt)
    #Step 2: rotation
    
    
    # Distance between (x1, y1) and (x2, y2)
    def dist(x1, x2, y1, y2):
        return ((x1-x2)**2+(y1-y2)**2)**0.5
    
    idx=0
    m=1
    # To find the index of coordinate with maximum y-coordinate
    if NumberPlateCnt is not None:
        for i in range(4):
            if NumberPlateCnt[i][0][1]>m:
                idx=i
                m=NumberPlateCnt[i][0][1]

        # Assign index to the previous coordinate
        if idx==0:
            pin=3
        else:
            pin=idx-1

        # Assign index to the next coordinate
        if idx==3:
            nin=0
        else:
            nin=idx+1

        # Find distances between the acquired coordinate and its previous and next coordinate
        p=dist(NumberPlateCnt[idx][0][0], NumberPlateCnt[pin][0][0], NumberPlateCnt[idx][0][1], NumberPlateCnt[pin][0][1])
        n=dist(NumberPlateCnt[idx][0][0], NumberPlateCnt[nin][0][0], NumberPlateCnt[idx][0][1], NumberPlateCnt[nin][0][1])

        # The coordinate that has more distance from the acquired coordinate is the required second bottom-most coordinate
        if p>n:
            if NumberPlateCnt[pin][0][0]<NumberPlateCnt[idx][0][0]:
                left=pin
                right=idx
            else:
                left=idx
                right=pin
            d=p
        else:
            if NumberPlateCnt[nin][0][0]<NumberPlateCnt[idx][0][0]:
                left=nin
                right=idx
            else:
                left=idx
                right=nin
            d=n

        left_x=NumberPlateCnt[left][0][0]
        left_y=NumberPlateCnt[left][0][1]
        right_x=NumberPlateCnt[right][0][0]
        right_y=NumberPlateCnt[right][0][1]
    #     print(left_x, left_y, right_x, right_y)

        # Finding the angle of rotation by calculating sin of theta
        opp=right_y-left_y
        hyp=((left_x-right_x)**2+(left_y-right_y)**2)**0.5
        sin=opp/hyp
        theta=math.asin(sin)*57.2958

        # Rotate the image according to the angle of rotation obtained
        image_center = tuple(np.array(ROI.shape[1::-1]) / 2)
        rot_mat = cv2.getRotationMatrix2D(image_center, theta, 1.0)
        result = cv2.warpAffine(ROI, rot_mat, ROI.shape[1::-1], flags=cv2.INTER_LINEAR)

        # The image can be cropped after rotation( since rotated image takes much more height)
        if opp>0:
            h=result.shape[0]-opp//2
        else:
            h=result.shape[0]+opp//2

        result=result[0:h, :]
    #     plt.imshow(result)
    #     plt.title("Plate obtained after rotation")
    #     plt.show()
        cv2.imshow("Plate obtained after rotation", result)

        return result

In [7]:
def find_contours(dimensions, img) :

    # Find all contours in the image
    cntrs, _ = cv2.findContours(img.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Retrieve potential dimensions
    lower_width = dimensions[0]
    upper_width = dimensions[1]
    lower_height = dimensions[2]
    upper_height = dimensions[3]
    
    # Check largest 5 or  15 contours for license plate or character respectively
    cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)[:15]
    
    ii = img
    
    x_cntr_list = []
    target_contours = []
    img_res = []
    for cntr in cntrs :
        # detects contour in binary image and returns the coordinates of rectangle enclosing it
        intX, intY, intWidth, intHeight = cv2.boundingRect(cntr)
        
        # checking the dimensions of the contour to filter out the characters by contour's size
        if intWidth > lower_width and intWidth < upper_width and intHeight > lower_height and intHeight < upper_height :
            x_cntr_list.append(intX) #stores the x coordinate of the character's contour, to used later for indexing the contours

            char_copy = np.zeros((44,24))
            # extracting each character using the enclosing rectangle's coordinates.
            char = img[intY:intY+intHeight, intX:intX+intWidth]
            char = cv2.resize(char, (20, 40))
            
            cv2.rectangle(ii, (intX,intY), (intWidth+intX, intY+intHeight), (50,21,200), 2)
#             plt.imshow(ii, cmap='gray')
#             plt.title('Predict Segments')
            cv2.imshow('prediction', ii)

            # Make result formatted for classification: invert colors
            char = cv2.subtract(255, char)

            # Resize the image to 24x44 with black border
            char_copy[2:42, 2:22] = char
            char_copy[0:2, :] = 0
            char_copy[:, 0:2] = 0
            char_copy[42:44, :] = 0
            char_copy[:, 22:24] = 0

            img_res.append(char_copy) # List that stores the character's binary image (unsorted)
            
    # Return characters on ascending order with respect to the x-coordinate (most-left character first)
            
    plt.show()
    # arbitrary function that stores sorted list of character indeces
    indices = sorted(range(len(x_cntr_list)), key=lambda k: x_cntr_list[k])
    img_res_copy = []
    for idx in indices:
        img_res_copy.append(img_res[idx])# stores character images according to their index
    img_res = np.array(img_res_copy)

    return img_res

def segment_characters(img_binary_lp) :
    if img_binary_lp is not None:
        LP_WIDTH = img_binary_lp.shape[0]
        LP_HEIGHT = img_binary_lp.shape[1]

        # Make borders white
        img_binary_lp[0:3,:] = 255
        img_binary_lp[:,0:3] = 255
        img_binary_lp[72:75,:] = 255
        img_binary_lp[:,330:333] = 255

        # Estimations of character contours sizes of cropped license plates
        dimensions = [LP_WIDTH/6,
                           LP_WIDTH/2,
                           LP_HEIGHT/10,
                           2*LP_HEIGHT/3]
        cv2.imwrite('contour.jpg',img_binary_lp)

        # Get contours within cropped license plate
        char_list = find_contours(dimensions, img_binary_lp)

        return char_list

# Predicting the output
def fix_dimension(img): 
    new_img = np.zeros((28,28,3))
    for i in range(3):
        new_img[:,:,i] = img
        return new_img
  
def show_results(char):
    dic = {}
    characters = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for i,c in enumerate(characters):
        dic[i] = c

    output = []
    for i,ch in enumerate(char): #iterating over the characters
        img_ = cv2.resize(ch, (28,28), interpolation=cv2.INTER_AREA)
        img = fix_dimension(img_)
        img = img.reshape(1,28,28,3) #preparing image for the model
        predict_x=loaded_model.predict(img) 
        classes_x=np.argmax(predict_x) #predicting the class
#         character = str(classes_x)
        character = dic[classes_x]
        output.append(character) #storing the result in a list
        
    plate_number = ''.join(output)
    
    return plate_number

In [8]:
def processingOCR(image):
    #01: Resizing
    if image is not None:
        resize_image = cv2.resize(image, None, fx=10, fy=10, interpolation=cv2.INTER_CUBIC)

        ## 02: Binarization 
        gray_image =  cv2.cvtColor(resize_image, cv2.COLOR_BGR2GRAY)

        ## 03: Noise Removal
        no_noise = cv2.bilateralFilter(gray_image, 11, 17, 17)

        #OG OG OG OG OG OG
    #     def noise_removal(image):
    #         import numpy as np
    #         kernel = np.ones((1, 1), np.uint8)
    #         image = cv2.dilate(image, kernel, iterations=1)
    #         kernel = np.ones((1, 1), np.uint8)
    #         image = cv2.erode(image, kernel, iterations=1)
    #         image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    #         image = cv2.medianBlur(image, 3)
    #         return (image)

    #     no_noise = noise_removal(gray_image)
        #OG OG OG OG OG OG


        ### 04: CLAHE
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl1 = clahe.apply(no_noise)

        ### 05: Contrast
        def contrasts(image):
            alpha = 1.5 # Contrast control (1.0-3.0)
            beta = 20 # Brightness control (0-100)

            return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

        contrasted_image = contrasts(cl1)

        ### 06: Erosion
        def thin_font(image):
            import numpy as np
            image = cv2.bitwise_not(image)
            kernel = np.ones((2,2),np.uint8)
            image = cv2.erode(image, kernel, iterations=1)
            image = cv2.bitwise_not(image)
            return (image)

        eroded_image = thin_font(contrasted_image)

    #     def thick_font(image):
    #         import numpy as np
    #         image = cv2.bitwise_not(image)
    #         kernel = np.ones((2,2),np.uint8)
    #         image = cv2.dilate(image, kernel, iterations=1)
    #         image = cv2.bitwise_not(image)
    #         return (image)

    #     dilated_image = thick_font(eroded_image)
    #     ax[1,2].imshow(dilated_image)
    #     ax[1,2].set_title('dilated_image')

        ### 07: Threshold
        thresh = cv2.threshold(eroded_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

        ### 08: Rotation and segmentation

        result = rotation_func(thresh)

        char_list = segment_characters(result)
        if char_list is not None:
            plate_no = show_results(char_list)
            print(plate_no)

    #     ### 08: Remove Border
    #     def remove_borders(image):
    #         contours, heiarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #         cntsSorted = sorted(contours, key=lambda x:cv2.contourArea(x))
    #         cnt = cntsSorted[-1]
    #         x, y, w, h = cv2.boundingRect(cnt)
    #         crop = image[y:y+h, x:x+w]
    #         return (crop)

    #     no_borders = remove_borders(thresh)
        cv2.imshow('LP', thresh)
    #     cv2.putText(image, plate_no, (50,50), cv2.FONT_ITALIC, 0.6, (0, 255, 0), 1)
    #     plt.imshow(no_borders)

    #     ax[1,4].imshow(no_borders)
    #     ax[1,4].set_title('Processed - Image')

In [9]:
from tensorflow import keras
loaded_model = keras.models.load_model('my_char_model.h5')

In [11]:
cap = cv2.VideoCapture('supercars_Trim.mp4')
# frames_counter = 1

# def rescale_frame(frame, percent=50):
#     width = int(frame.shape[1] * percent/ 100)
#     height = int(frame.shape[0] * percent/ 100)
#     dim = (width, height)
#     return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

while cap.isOpened():
    ret, frame = cap.read()
#     frames_counter = frames_counter + 1
    
#     rescale = rescale_frame(frame)
    # Make detections 
    results = yolo_model(frame)
#     results = yolo_model(rescale)q
    res = results.pandas().xyxy[0].sort_values('xmin')
    # print(!res.empty)
    if not (res.empty) and res.loc[0,'confidence'] >=0.70:
        # print("Current Frame has LP and atleast one confidence is high enough")
        # print(res)
        X1 = int(res.loc[0,'xmin'])
        Y1 = int(res.loc[0,'ymin'])
        X2 = int(res.loc[0,'xmax'])
        Y2 = int(res.loc[0,'ymax'])
        roi = frame[Y1-5:Y2+5, X1-5:X2+5]
        processingOCR(roi)
        # cv2.imshow("License Plate", roi)
        # crops = results.crop(save=False)
    cv2.imshow('YOLO', np.squeeze(results.render()))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
# print("Number of frames in the video: ", frames_counter)
cap.release()
cv2.destroyAllWindows()

UP6CXZ89S
UP6CXZ89S
UP6CXZ89S
UP6CXZ89S
U6CXZ89S
U6CI28SS
U6CXZ8S5


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


## CNN character recognition

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255, width_shift_range=0.1, height_shift_range=0.1)
path = 'data/data'
train_generator = train_datagen.flow_from_directory(
        path+'/train',  # this is the target directory
        target_size=(28,28),  # all images will be resized to 28x28
        batch_size=1,
        class_mode='sparse')

validation_generator = train_datagen.flow_from_directory(
        path+'/val',  # this is the target directory
        target_size=(28,28),  # all images will be resized to 28x28 batch_size=1,
        class_mode='sparse')

Found 864 images belonging to 36 classes.
Found 216 images belonging to 36 classes.


In [13]:
K.clear_session()
model = Sequential()
model.add(Conv2D(16, (22,22), input_shape=(28, 28, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (16,16), input_shape=(28, 28, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (8,8), input_shape=(28, 28, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (4,4), input_shape=(28, 28, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(36, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.Adam(lr=0.0001), metrics='accuracy')

D:\Anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        23248     
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        131104    
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        131136    
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 64)        65600     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 7, 64)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 7, 7, 64)          0         
                                                        

In [15]:
batch_size = 1
result = model.fit(
      train_generator,
      steps_per_epoch = train_generator.samples // batch_size,
      validation_data = validation_generator, 
      epochs = 25, verbose=1, callbacks=None)

Epoch 1/25
219/864 [======>.......................] - ETA: 19s - loss: 3.5843 - accuracy: 0.0457

KeyboardInterrupt: 

In [ ]:
model.save("my_char_model.h5")

In [ ]:
fig = plt.figure(figsize=(14,5))
grid=gridspec.GridSpec(ncols=2,nrows=1,figure=fig)
fig.add_subplot(grid[0])
plt.plot(result.history['accuracy'], label='training accuracy')
plt.plot(result.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

fig.add_subplot(grid[1])
plt.plot(result.history['loss'], label='training loss')
plt.plot(result.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

In [ ]:
# Create a new model instance
loaded_model = Sequential()
loaded_model.add(Conv2D(16, (22,22), input_shape=(28, 28, 3), activation='relu', padding='same'))
loaded_model.add(Conv2D(32, (16,16), input_shape=(28, 28, 3), activation='relu', padding='same'))
loaded_model.add(Conv2D(64, (8,8), input_shape=(28, 28, 3), activation='relu', padding='same'))
loaded_model.add(Conv2D(64, (4,4), input_shape=(28, 28, 3), activation='relu', padding='same'))
loaded_model.add(MaxPooling2D(pool_size=(4, 4)))
loaded_model.add(Dropout(0.4))
loaded_model.add(Flatten())
loaded_model.add(Dense(128, activation='relu'))
loaded_model.add(Dense(36, activation='softmax'))

# Restore the weights
loaded_model.load_weights('checkpoints/my_checkpoint')